In [90]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random

In [52]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shanm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
#loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('train.csv')
news_dataset.shape
#20800 news article

(20800, 5)

In [10]:
news_dataset.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
#if there are so many nulls then we have to drop or replace with null string
#lot of values are missing we can use imputation method
#if we have large data we can simply drop




In [12]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [13]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [14]:
#merging the author name and news title
news_dataset['content' ] = news_dataset['author']+' '+news_dataset['title']

In [15]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [58]:
#seperating the data and the label
x = news_dataset.drop(columns='label',axis = 1)
#if we remove the column we have to mention the axis = 1
#if we remove the row we have to mention the axis = 0
y = news_dataset['label']

In [59]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [60]:
#stemming 
#stemming is the process of reducing a word to its root word
#example
#actor , actress , acting -- > act main root word will be taken 

In [18]:
port_stem = PorterStemmer()

In [61]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [62]:
news_dataset['content']= news_dataset['content'].apply(stemming)

In [63]:
print(news_dataset['content'])



0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [64]:
x = news_dataset['content'].values
y = news_dataset['label'].values

In [68]:
print(x)
print(y)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


converting the text to the numerical value using tfidfvectorizer

In [71]:
vectorizer = TfidfVectorizer()

vectorizer.fit(x)
x = vectorizer.transform(x)

In [72]:
print(x)

  (0, 15545)	0.28485063562728646
  (0, 13342)	0.2565896679337957
  (0, 8818)	0.3635963806326075
  (0, 8542)	0.29212514087043684
  (0, 7608)	0.24785219520671603
  (0, 6928)	0.21874169089359144
  (0, 4916)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16646)	0.3025156488372128
  (1, 6742)	0.19152496072048605
  (1, 5439)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15470)	0.41544962664721613
  (2, 9528)	0.49351492943649944
  (2, 5901)	0.3474613386728292
  (2, 5325)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12992)	0.2483705036831893
  (20797, 12222)	0.27276402145717243
  (20797, 12017)	0.24790022252744132
  (20797, 10212)	0.0804189541935242
  (20797, 9496)	0.17463635692029988
  (20797, 9427)	0.29394

In [73]:
#splitting the dataset to training and test data

In [74]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [77]:
print(x_train)
print(y_train)

  (0, 16330)	0.4996597593503279
  (0, 5189)	0.4463436441397709
  (0, 5078)	0.4119504446957197
  (0, 4929)	0.4971225616226277
  (0, 2491)	0.36644708494236555
  (1, 16843)	0.08876491888832169
  (1, 15442)	0.11047319384656958
  (1, 15156)	0.08709549399046825
  (1, 14441)	0.33000273429222493
  (1, 11780)	0.2972483036435648
  (1, 11047)	0.22582299186169705
  (1, 10967)	0.26814459214916153
  (1, 10212)	0.08580194257183664
  (1, 9014)	0.413993837845734
  (1, 8934)	0.24097470973549523
  (1, 6362)	0.24181915546499674
  (1, 2592)	0.25069021478042075
  (1, 2469)	0.31362551896789487
  (1, 2202)	0.21270007149358267
  (1, 814)	0.29102197534690305
  (1, 263)	0.2563885462401546
  (2, 16843)	0.08149322997352774
  (2, 15722)	0.3393400733660584
  (2, 15442)	0.10142314672055713
  (2, 15156)	0.07996056561886902
  :	:
  (16637, 9013)	0.259342820142701
  (16637, 8401)	0.403404675895816
  (16637, 6833)	0.4558039677254956
  (16637, 6828)	0.3141274035345144
  (16638, 15434)	0.26315079262300584
  (16638, 14468)	

In [78]:
#training the model : using logistic regression
model = LogisticRegression()

In [79]:
model.fit(x_train,y_train)

LogisticRegression()

In [80]:
#Evaluation #accuracy score of training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)


In [81]:
print(training_data_accuracy)

0.9872596153846154


In [82]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)

In [83]:
#this is the testing data accuracy
print(test_data_accuracy)

0.9774038461538461


In [104]:
#BUILDING THE PREDICTIVE SYSTEM
anynum = random.randint(0,1300)
print(anynum)
x_newnews = x_test[anynum]
prediction = model.predict(x_newnews)

if(prediction):
    print("THE NEWS IS REAL")
else:
    print("The NEWS IS FAKE")


393
THE NEWS IS REAL


THE NEWS IS REAL
